# Google Colab 運行 `text2srt_tts` 專案

這個筆記本將引導您如何在 Google Colab 環境中運行 `text2srt_tts` 專案，並透過 `ngrok` 建立一個公開的 Streamlit 介面連結。
安裝專案所需的 Python 依賴套件，包括 `streamlit`, `azure-cognitiveservices-speech`, `edge-tts`, `pydub`, `tqdm`, `pytest` 以及 `pyngrok` (用於建立公開連結)。


In [ ]:
!git clone https://github.com/your-username/text2srt_tts.git
%cd text2srt_tts

# 安裝必要的Python套件
!pip install -q -r requirements.txt
!pip install pyngrok --quiet

print("✅ 套件安裝完成！")

## 啟動 Streamlit 應用程式並使用 ngrok 建立公開連結

我們將使用 `pyngrok` 來建立一個安全的隧道，將 Colab 中運行的 Streamlit 應用程式暴露到公共網路。您需要一個 `ngrok` 認證令牌 (Auth Token)。

1.  前往 [ngrok 官網](https://ngrok.com/) 註冊一個免費帳號。
2.  登入後，在您的 Dashboard 找到您的 Auth Token (通常在 "Your Authtoken" 頁面)。
3.  將您的 Auth Token 貼到下面的 `NGROK_AUTH_TOKEN` 變數中。


In [ ]:
from pyngrok import ngrok
import subprocess
import threading
import time

# 終止所有現有的 ngrok 通道
ngrok.kill()

# 請替換為您的 ngrok Auth Token
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN"

# 設置 ngrok 認證
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Streamlit 應用程式的端口
streamlit_port = 8501

# 啟動 Streamlit 應用程式的函數
def start_streamlit():
    print("啟動 Streamlit 應用程式...")
    # 使用 -m 參數確保模組正確載入
    subprocess.run(["python", "-m", "streamlit", "run", "app/ui/gui.py", "--server.port", str(streamlit_port), "--server.enableCORS", "false", "--server.enableXsrfProtection", "false"])

# 在獨立的執行緒中啟動 Streamlit
streamlit_thread = threading.Thread(target=start_streamlit)
streamlit_thread.daemon = True # 設置為守護執行緒，當主程式結束時自動終止
streamlit_thread.start()

time.sleep(5) # 等待 Streamlit 啟動

# 建立 ngrok 隧道
print("建立 ngrok 隧道...")
public_url = ngrok.connect(streamlit_port)
print(f"您的 Streamlit 應用程式已在以下公開連結運行: {public_url}")

# 心跳機制：保持 Colab 會話活躍
# 運行此循環將阻止 Colab 會話因閒置而斷開，直到您手動停止執行。
try:
    while True:
        print("心跳中... Colab 會話保持活躍。")
        time.sleep(60) # 每分鐘發送一次心跳
except KeyboardInterrupt:
    print("心跳機制已停止。")
    ngrok.kill()
    print("ngrok 隧道已關閉。")
